In [1]:
#Python libraries
from pyomo.environ import * 
# Pyomo library for solving a problem
from pyomo.opt import SolverFactory
# Pyomo library for checking the solver statuse 
from pyomo.opt import SolverStatus, TerminationCondition
#panda is a useful data manipulation library
import numpy as np
import pandas as pd
import networkx as nx
import pylab as plt
from scipy import sparse
from sys import getsizeof

In [2]:
#Homemade functions
from getParam import getParam
from getDemand import getDemand
from objRule import objRule
from flowBalRule import flowBalRule
from flowLinkRule import flowLinkRule
from costLinkRule import costLinkRule


filenameNet = "Data/SiouxFalls/SiouxFalls_net.tntp"

param,A,M,N,L = getParam(filenameNet)

filenameTrip = "Data/SiouxFalls/SiouxFalls_trips.tntp"

d, OD_pair = getDemand(filenameTrip,L)

{0: [(0, 1), (1, 1), (2, -1), (4, -1)],
 1: [(0, -1), (2, 1), (3, 1), (13, -1)],
 2: [(1, -1), (4, 1), (5, 1), (6, 1), (7, -1), (34, -1)],
 3: [(5, -1), (7, 1), (8, 1), (9, 1), (10, -1), (30, -1)],
 4: [(8, -1), (10, 1), (11, 1), (12, 1), (14, -1), (22, -1)],
 5: [(3, -1), (11, -1), (13, 1), (14, 1), (15, 1), (18, -1)],
 6: [(16, 1), (17, 1), (19, -1), (53, -1)],
 7: [(15, -1),
  (16, -1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (23, -1),
  (46, -1)],
 8: [(12, -1), (20, -1), (22, 1), (23, 1), (24, 1), (25, -1)],
 9: [(24, -1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (31, -1),
  (42, -1),
  (47, -1),
  (50, -1)],
 10: [(9, -1),
  (26, -1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (35, -1),
  (39, -1)],
 11: [(6, -1), (32, -1), (34, 1), (35, 1), (36, 1), (37, -1)],
 12: [(36, -1), (37, 1), (38, 1), (73, -1)],
 13: [(33, -1), (39, 1), (40, 1), (41, 1), (43, -1), (70, -1)],
 14: [(27, -1),
  (40, -1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (56, -1),
  (6

In [3]:
model=ConcreteModel()
# model.links=Set(initialize = (a[0]+1 for a in A))
model.links=RangeSet(0,len(A)-1,1)
model.od=RangeSet(0,M.shape[1]-1,1)
model.nodes=RangeSet(0,M.shape[0]-1,1)
model.f=Var(model.links,within=NonNegativeReals)
model.fk=Var(model.links,model.od,within=NonNegativeReals)
model.c=Var(model.links,within=NonNegativeReals)
# model.t0 = Param(initialize=param['t_0'])

AttributeError: 'dict' object has no attribute 'shape'

In [ ]:
model.obj = Objective(expr=objRule(model))

In [ ]:
model.pprint()

In [ ]:
model.flowBal =  ConstraintList()
for i in model.nodes:
    for k in model.od:
        model.flowBal.add(flowBalRule(model,d,M,i,k))
#Constraint(model.nodes,model.od, expr=flowBalRule(model,d,M,i,k) for i in model.nodes for k in model.od)

In [ ]:
model.flowLink =  ConstraintList()
model.costLink = ConstraintList()
for a in model.links:
    model.flowLink.add(flowLinkRule(model,a))
    model.costLink.add(costLinkRule(model,param,a))
#model.flowLink = Constraint(model.links, rule=flowLinkRule)

In [ ]:
model.pprint()

In [ ]:
opt = SolverFactory("ipopt")

#opt.options["mipgap"] = 0.05
#store the results 
results = opt.solve(model)
print(results)

In [ ]:
model.display()